In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Feel free to use any other packages combination satesfying the compatibility matrix provided above.
packages = [
    "com.clickhouse.spark:clickhouse-spark-runtime-3.4_2.12:0.8.0",
    "com.clickhouse:clickhouse-client:0.7.0",
    "com.clickhouse:clickhouse-http-client:0.7.0",
    "org.apache.httpcomponents.client5:httpclient5:5.2.1"

]

spark = (SparkSession.builder
         .config("spark.jars.packages", ",".join(packages))
         .getOrCreate())

spark.conf.set("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
spark.conf.set("spark.sql.catalog.clickhouse.host", "clickhouse-server")
spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
spark.conf.set("spark.sql.catalog.clickhouse.password", "1234qwe")
spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.conf.set("spark.clickhouse.write.format", "json")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-client added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b755c253-c044-4421-9ed8-ae9f475a51a9;1.0
	confs: [default]
	found com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-client;0.7.0 in central
	found com.clickhouse#clickhouse-data;0.7.0 in central
	found com.clickhouse#clickhouse-http-client;0.7.0 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;1.7.36 in central
down

In [2]:
spark.sql("""
CREATE TABLE IF NOT EXISTS clickhouse.default.example_table (
    id LONG NOT NULL,
    name STRING
)
USING clickhouse
TBLPROPERTIES (
    'engine'='MergeTree()',
    'order_by'='id'
)
""")

DataFrame[]

In [3]:
# Create DataFrame
data = [Row(id=11, name="John"), Row(id=12, name="Doe")]
df = spark.createDataFrame(data)

# Write DataFrame to ClickHouse
df.writeTo("clickhouse.default.example_table").append()

In [4]:
df = spark.sql("select * from clickhouse.default.example_table")
df.show()

+---+----+
| id|name|
+---+----+
| 12| Doe|
| 11|John|
| 12| Doe|
| 11|John|
+---+----+

